In [10]:
import os
import sqlite3 as sql
from datetime import datetime

import pandas as pd

# Coleta de dados do site do IBGE

In [11]:
# Coletando dados do IBGE
url = 'https://www.ibge.gov.br/explica/codigos-dos-municipios.php#RJ'

dadosIBGE = pd.read_html(url, match="Municípios do Rio de Janeiro")[0]

# Renomear colunas do df
dadosIBGE = dadosIBGE.rename(columns={'Municípios do Rio de Janeiro':'nmMunic','Códigos':'codMunic'})

# Alterando o nome do index
dadosIBGE.index.name = 'idMunic'

# Alterando o index para comelar em 1
dadosIBGE.index = dadosIBGE.index + 1

# Adicionoar uma coluna de data e hora decarga 
dtCarga = datetime.today().strftime('%d/%m/%Y %H:%M')
dadosIBGE['dtCarga'] = dtCarga

# dadosIBGE

# Criando o Banco de Dados ODS

In [12]:
# Manipulando o sistema de arquivos
path = os.getcwd()

BDODS = os.path.join(path, "pascoaODS.db")
BDDW = os.path.join(path, "pascoaDW.db")

# Manipulando os Bancos de Dados Criados

In [13]:
# Conectar no BDODS
conexaoODS = sql.connect(BDODS)

# Criar a tabela tbLogMunic e carregar os dados do DF dadosIBGE
dadosIBGE.to_sql('tbLogMunic', conexaoODS, if_exists="append")

# Confirmar a transação
conexaoODS.commit()

# Fechat a conexão
conexaoODS.close()

print("Cargo do BDODS concluída!")

Cargo do BDODS concluída!


In [14]:
# Conectar no BDDW
conexaoDW = sql.connect(BDDW)

# Selecionar somente as colunas para criação da dMunicipio
dadosIBGE = dadosIBGE[['nmMunic', 'codMunic']]

# Criar a tabela dMunicipio e carregar os dados do DF dadosIBGE
dadosIBGE.to_sql('dMunicipio', conexaoDW, if_exists="replace")

# Confirmar a transação
conexaoDW.commit()

# Fechat a conexão
conexaoDW.close()

print("Cargo do BDDW concluída!")

Cargo do BDDW concluída!


# Carga dos Dados de Cadastro das Delegacias de Polícia

In [15]:
# Definir o endereço fisico do arquivo CSV
pathDP  = os.path.join(path, 'DP.csv')

# Cria DataFrame com os dados das DPs
tbLogDP = pd.read_csv(pathDP)

# Criando coluna com a data da carga
tbLogDP['dtCarga'] = dtCarga
# Selecionado a coluna COD_DP como novo index
# tbLogDP = tbLogDP.set_index("COD_DP")

# Carga dos dados de cadastro dos Responsaveis pelas delegacia de policia

In [16]:
# Criar DataFrame com os dados com os reponsaveis pelas DPs
path_responsaveis = os.path.join(path, 'ResponsavelDP.csv')

tbLogRespDP = pd.read_csv(path_responsaveis)

# Criando coluna com a data da carga
tbLogRespDP['dtCarga'] = dtCarga

# Selecionado a coluna COD_DP como novo index
# tbLogRespDP = tbLogRespDP.set_index("COD_DP")

# Conectando no ODS

In [17]:
# Conectando no ODS
conexaoODS = sql.connect(BDODS)

# Definição de manipulação de dados em SQL
sql_ODS = conexaoODS.cursor()


# Inserindo Dados das Delegacias de Polícia, no SQLite

In [18]:
# Inserindo registros na tabela tbLogDP
print("Iniciando a inserção de dados na tabela tbLogDP.")
print("------------------------------------------------")

sql_ODS.executemany(''' INSERT INTO tbLogDP (codDP, nmDP, endereco, dtCarga) VALUES (?,?,?,?)''',tbLogDP.values.tolist()) 

# Confirmar a transação
conexaoODS.commit()

# Exibir uma mensagem de conclusão de carga
print(f"Carga Finalizada! {len(tbLogDP)} registros inseridos na tbLogDP")

Iniciando a inserção de dados na tabela tbLogDP.
------------------------------------------------
Carga Finalizada! 137 registros inseridos na tbLogDP


# Inserindo Dados dos Responsaveis pela Delegacias de Polícia, no SQLite


In [19]:
# Inserindo registros na tabela tbLogRespDP
print("Iniciando a inserção de dados na tabela tbLogRespDP.")
print("------------------------------------------------")

sql_ODS.executemany(''' INSERT INTO tbLogRespDP (codDP, nmResponsavel, dtCarga) VALUES (?,?,?)''',tbLogRespDP.values.tolist()) 

# Confirmar a transação
conexaoODS.commit()

# Exibir uma mensagem de conclusão de carga
print(f"Carga Finalizada! {len(tbLogRespDP)} registros inseridos na tbLogRespDP")

Iniciando a inserção de dados na tabela tbLogRespDP.
------------------------------------------------
Carga Finalizada! 137 registros inseridos na tbLogRespDP


# Validando Carga dos Dados de DP e de Responsável do DP

In [20]:
# Validadar a carga através de um select na tbLogDP
# pd.read_sql("SELECT * FROM tbLogDP", conexaoODS)

In [21]:
# Validadar a carga através de um select na tbLogRespDP
# pd.read_sql("SELECT * FROM tbLogRespDP", conexaoODS)

# Criando o DataFrame com o select de criação da dimensão DP (dDP), no DW

In [22]:
# definir a query de criação da dimensão dDp

qry_dDP = '''
    SELECT
        codDP,
        nmDP,
        endereco,
        nmResponsavel
    FROM
    (
        SELECT
            a.codDP,
            a.nmDP,
            a.endereco,
            b.nmResponsavel,
            MAX(a.dtCarga)
        FROM  tbLogDP a
        JOIN tbLogRespDP b ON (a.codDP  = b.codDP)
        WHERE a.dtCarga = (
		    SELECT MAX(x.dtCarga)
		    FROM tbLogDP x
	    )
        GROUP BY 
            a.codDP,
            a.nmDP,
            a.endereco,
            b.nmResponsavel
    ) a
'''

dDP = pd.read_sql(qry_dDP, conexaoODS)

# Coletando dados dos Batalhões de Polícia Militar (BPM)

In [23]:
# Criar DataFrame com os dados dos BPMs
path_BPM = os.path.join(path, 'BPM.csv')
tbLogBPM = pd.read_csv(path_BPM)

# Criando coluna com a data da carga
tbLogBPM['dtCarga'] = dtCarga

# Construindo a Tebela de Log de Registros do BPM (tbLogBPM) no ODS

In [24]:
# Definir a query decriação da tabela
qry_tbLogBPM = """
    CREATE TABLE IF NOT EXISTS tbLogBPM(
    
    codBPM INTEGER,
    nmBPM VARCHAR (7),
    enderecoBPM VARCHAR (200),
    dtCarga DATETIME
)
"""

# executar a criação da tbLogBPM
sql_ODS.execute(qry_tbLogBPM)

# definição da qry de criação de index
tbLogBPM_codBPM_IDX = "CREATE INDEX IF NOT EXISTS tbLogBPM_codBPM_IDX ON tbLogBPM (codBPM)"

# executar a criação do index
sql_ODS.execute(tbLogBPM_codBPM_IDX)


# Inserindo Dados de BPM na Tabela tbLogBPM

In [25]:
# Inserindo registros na tabela tbLogBPM
print("Iniciando a inserção de dados na tabela tbLogBPM.")
print("------------------------------------------------")

sql_ODS.executemany(''' INSERT INTO tbLogBPM (codBPM, nmBPM, enderecoBPM, dtCarga) VALUES (?,?,?,?)''',tbLogBPM.values.tolist()) 

# Confirmar a transação
conexaoODS.commit()

# Exibir uma mensagem de conclusão de carga
print(f"Carga Finalizada! {len(tbLogBPM)} registros inseridos na tbLogBPM")

Iniciando a inserção de dados na tabela tbLogBPM.
------------------------------------------------


Carga Finalizada! 39 registros inseridos na tbLogBPM


# Coletando dados dos Batalhões de Polícia Militar (BPM)

In [26]:
# Criar DataFrame com os dados dos BPMs
path_AreaBPM = os.path.join(path, 'areaBPM.csv')
tbLogAreaBPM = pd.read_csv(path_AreaBPM)

# Criando coluna com a data da carga
tbLogAreaBPM['dtCarga'] = dtCarga

# Construindo a Tebela de Log de Registros da Área dos BPM (tbLogAreaBPM) no ODS

In [27]:
# Definir a query decriação da tabela
qry_tbLogAreaBPM= """
    CREATE TABLE IF NOT EXISTS tbLogAreaBPM(
    
    codBPM INTEGER,
    areaBPM REAL (5,2),
    dtCarga DATETIME
)
"""

# executar a criação da tbLogAreaBPM
sql_ODS.execute(qry_tbLogAreaBPM)

# definição da qry de criação de index
tbLogAreaBPM_codBPM_IDX = "CREATE INDEX IF NOT EXISTS tbLogAreaBPM_codBPM_IDX ON tbLogAreaBPM (codBPM)"

# executar a criação do index
sql_ODS.execute(tbLogAreaBPM_codBPM_IDX)


# Inserindo Dados da Área dos BPM na Tabela tbLogAreaBPM

In [28]:
# Inserindo registros na tabela tbLogBPM
print("Iniciando a inserção de dados na tabela tbLogAreaBPM.")
print("------------------------------------------------")

sql_ODS.executemany(''' INSERT INTO tbLogAreaBPM (codBPM, areaBPM, dtCarga) VALUES (?,?,?)''',tbLogAreaBPM.values.tolist()) 

# Confirmar a transação
conexaoODS.commit()

# Exibir uma mensagem de conclusão de carga
print(f"Carga Finalizada! {len(tbLogAreaBPM)} registros inseridos na tbLogAreaBPM")

Iniciando a inserção de dados na tabela tbLogAreaBPM.
------------------------------------------------
Carga Finalizada! 39 registros inseridos na tbLogAreaBPM


# Criando o DataFrame com o select de criação da dimensão BPM (dBPM), no DW

In [29]:
# definir a query de criação da dimensão dBPM

qry_dBPM = '''
SELECT
	codBPM,
	nmBPM,
	enderecoBPM,
	areaBPM
FROM (
	SELECT 
		a.codBPM,
		a.nmBPM,
		a.enderecoBPM,
		b.areaBPM,
		MAX(a.dtCarga)
	FROM tbLogBPM a
	JOIN tbLogAreaBPM b ON a.codBPM = b.codBPM
	WHERE a.dtCarga = (
		SELECT MAX(x.dtCarga)
		FROM tbLogBPM x
	)
	GROUP BY
		a.codBPM,
		a.nmBPM,
		a.enderecoBPM,
		b.areaBPM
)
'''

dBPM = pd.read_sql(qry_dBPM, conexaoODS)

# Fecando a conexão com o ODS

In [30]:
# Confirmar a transação 
conexaoODS.commit()

# Fechar a conexão
conexaoODS.close()

# Conectando no DW e Informando ao python que usarei Linguagem SQL

In [31]:
# Conectar no BDDW
conexaoDW = sql.connect(BDDW)

# Definição de manipulação de dados em SQL
sql_DW = conexaoDW.cursor()

# Deletando os Dados Antigos e Reiniciando o Autoincremental da Chave, da Tabela dDP

In [32]:
# Deletar os dados atuais
sql_DW.execute("DELETE FROM dDP") # A alegria de todo desenvolvedor

# Reiniciando o autoincremental da Chave
sql_DW.execute("UPDATE sqlite_sequence SET seq=0 WHERE name='dDP'")

# Confirmar a transação
conexaoDW.commit()


# Inserido dados na Dimensão DP (dDP), no DW

In [33]:
# Inserindo registros na tabela dDP
print("Iniciando a inserção de dados na tabela dDP.")
print("------------------------------------------------")

sql_DW.executemany(''' INSERT INTO dDP (codDP, nmDP, endereco, nmResponsavel) VALUES (?,?,?,?)''',dDP.values.tolist()) 

# Confirmar a transação
conexaoDW.commit()

# Exibir uma mensagem de conclusão de carga
print(f"Carga Finalizada! {len(dDP)} registros inseridos na dDP")

Iniciando a inserção de dados na tabela dDP.
------------------------------------------------
Carga Finalizada! 137 registros inseridos na dDP


# Validando Carga  dos Dados da Dimensão DP (dDP)

In [34]:
# Validadar a carga através de um select na dDP
# pd.read_sql("SELECT * FROM dDP", conexaoDW)

# Construindo a Tebela de Dimensão BPM (dBPM) no DW

In [35]:
# Definir a query decriação da tabela
qry_dBPM= """
    CREATE TABLE IF NOT EXISTS dBPM(
    
    idBPM INTEGER PRIMARY KEY AUTOINCREMENT,
    codBPM INTEGER,
    nmBPM VARCHAR (7),
    enderecoBPM VARCHAR (200),
    areaBPM REAL (5,2)
)
"""

# executar a criação da dBPM
sql_DW.execute(qry_dBPM)

# definição da qry de criação de index idBPM
dBPM_idBPM_IDX = "CREATE INDEX IF NOT EXISTS dBPM_idBPM_IDX ON dBPM (idBPM)"

# definição da qry de criação de index codBPM
dBPM_codBPM_IDX = "CREATE INDEX IF NOT EXISTS dBPM_codBPM_IDX ON dBPM (codBPM)"

# executar a criação do index idBPM
sql_DW.execute(dBPM_idBPM_IDX)

# executar a criação do index codBPM
sql_DW.execute(dBPM_codBPM_IDX)

# Confirmar a transação
conexaoDW.commit()

# Construindo a Tebela de Dimensão Periodo (dPeriodo) no DW

In [36]:
# Definir a query decriação da tabela
qry_dPeriodo= """
    CREATE TABLE IF NOT EXISTS dPeriodo(
    
    idPeriodo INTEGER PRIMARY KEY AUTOINCREMENT,
    data DATETIME,
    mes INTERGER,
    ano INTEGER,
    trimestre INTEGER,
    semestre INTEGER
)
"""

# executar a criação da dPeriodo
sql_DW.execute(qry_dPeriodo)

# definição da qry de criação de index idPeriodo
dPeriodo_idPeriodo_IDX = "CREATE INDEX IF NOT EXISTS dPeriodo_idPeriodo_IDX ON dPeriodo (idPeriodo)"

# definição da qry de criação de index codPeriodo
dPeriodo_IDX = "CREATE INDEX IF NOT EXISTS dPeriodo_IDX ON dPeriodo (mes, ano, trimestre,semestre)"

# executar a criação do index idPeriodo
sql_DW.execute(dPeriodo_idPeriodo_IDX)

# executar a criação do index dPeriodo_IDX
sql_DW.execute(dPeriodo_IDX)

# Confirmar a transação
conexaoDW.commit()

# Deletando os Dados Antigos e Reiniciando o Autoincremental da Chave, da Tabela dBPM 

In [37]:
# Deletar os dados atuais
sql_DW.execute("DELETE FROM dBPM") # A alegria de todo desenvolvedor

# Reiniciando o autoincremental da Chave
sql_DW.execute("UPDATE sqlite_sequence SET seq=0 WHERE name='dBPM'")

# Confirmar a transação
conexaoDW.commit()


# Inserido dados na Dimensão BPM (dBPM), no DW

In [38]:
# Inserindo registros na tabela dBPM
print("Iniciando a inserção de dados na tabela dBPM.")
print("------------------------------------------------")

sql_DW.executemany(''' INSERT INTO dBPM (codBPM, nmBPM, enderecoBPM, areaBPM) VALUES (?,?,?,?)''',dBPM.values.tolist()) 

# Confirmar a transação
conexaoDW.commit()

# Exibir uma mensagem de conclusão de carga
print(f"Carga Finalizada! {len(dBPM)} registros inseridos na dBPM")

Iniciando a inserção de dados na tabela dBPM.
------------------------------------------------
Carga Finalizada! 39 registros inseridos na dBPM


# Fechando a conexão com o DW

In [39]:
# Confirmar a transação
conexaoDW.commit()

# Fechar a conexão
conexaoDW.close()
